In [1]:
# Main notebook for playing around looking at different things in data
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
plt.rcParams['lines.markersize'] = 10
plt.rcParams['lines.linewidth'] = 3
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [2]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath

In [3]:
dfAdm = pd.read_excel('Admitted\Admitted.xlsx')
dfAdm = dfAdm.transpose()
dfAdm.columns = dfAdm.iloc[0]
dfAdm = dfAdm.drop(['Aldersgruppe']) 
curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')

In [4]:
dfAdmDiff = dfAdm.diff().iloc[1:]
plotDates = curDates[1:]

In [5]:
dfCase = pd.DataFrame(columns=dfAdm.columns)
dfTest = pd.DataFrame(columns=dfAdm.columns)
dfPosP = pd.DataFrame(columns=dfAdm.columns)

In [6]:
# Collect the cases, tests and positive percentage from all the "cases by age" files

ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath

for subdir, dirs, files in os.walk(rootdir):
    if not len(files) == 0:
        latestdir = subdir
        latestDate = pd.to_datetime(subdir[-10:])
        curdf = pd.read_csv(latestdir+'/Cases_by_age.csv',delimiter = ';',dtype=str)
        
        curdf['Antal_bekræftede_COVID-19'] = pd.to_numeric(curdf['Antal_bekræftede_COVID-19'].astype(str).apply(lambda x: x.replace('.','')))
        curdf['Antal_testede'] = pd.to_numeric(curdf['Antal_testede'].astype(str).apply(lambda x: x.replace('.','')))
        curdf['Procent_positive'] = pd.to_numeric(curdf['Procent_positive'].astype(str).apply(lambda x: x.replace(',','.')))
        
        
        dfCase.loc[latestDate] = curdf['Antal_bekræftede_COVID-19'].values
        dfTest.loc[latestDate] = curdf['Antal_testede'].values
        dfPosP.loc[latestDate] = curdf['Procent_positive'].values

allDates = dfCase.index


In [11]:
# dfCaseDiff.tail()

In [12]:
dfCaseDiff = dfCase.diff().iloc[1:]
dfTestDiff = dfTest.diff().iloc[1:]
dfPosPDiff = dfPosP.diff().iloc[1:]
plotDatesCase = allDates[1:]

In [13]:
fig,ax1 = plt.subplots(1,1)

howManyToUse = len(dfCaseDiff.columns)-4
cmap = plt.cm.get_cmap('Dark2',howManyToUse)
for k in range(0,howManyToUse):
    curColName = dfCaseDiff.columns[k]
    ax1.plot(plotDatesCase,dfCaseDiff[curColName],'.-',linewidth=0.4,label=curColName,color=cmap(k))
    # meanWidth = 7
    # ax1.plot(rnTime(plotDatesCase,meanWidth),rnMean(dfCaseDiff[curColName],meanWidth),label=curColName,color=cmap(k))


testMonday = np.datetime64('2021-05-17')
numMondays = 30
for m in range(0,numMondays):
    ax1.bar(testMonday-np.timedelta64(m*7,'D'),1000,color='grey')

ax1.legend()

ax1.set_ylim(bottom=0,top=1000)
# ax1.set_xlim([np.datetime64('2021-03-01'),np.datetime64('2021-05-05')])
ax1.set_xlim(left=np.datetime64('2021-03-01'))

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …